In [1]:
from pathlib import Path
import json
import csv
import os
import matplotlib.pyplot as plt
import numpy as np

In [4]:
json_files = 'E:/SJ/cricket/analysis/IPL/json'

# Baby Steps
purpose here is to understand how to work with the JSON files and extract data

### Example File

In [10]:
with open(r'E:\SJ\cricket\analysis\json\1304050.json') as f:
    eg_file = json.loads(f.read())

In [13]:
eg_file.keys()

dict_keys(['meta', 'info', 'innings'])

In [14]:
eg_file['meta'].keys()

dict_keys(['data_version', 'created', 'revision'])

In [12]:
eg_file['info'].keys()

dict_keys(['balls_per_over', 'city', 'dates', 'event', 'gender', 'match_type', 'officials', 'outcome', 'overs', 'player_of_match', 'players', 'registry', 'season', 'team_type', 'teams', 'toss', 'venue'])

In [91]:
eg_file['info']['players']

{'Lucknow Super Giants': ['KL Rahul',
  'Q de Kock',
  'E Lewis',
  'MK Pandey',
  'DJ Hooda',
  'A Badoni',
  'KH Pandya',
  'PVD Chameera',
  'Mohsin Khan',
  'Ravi Bishnoi',
  'Avesh Khan'],
 'Gujarat Titans': ['Shubman Gill',
  'MS Wade',
  'V Shankar',
  'HH Pandya',
  'DA Miller',
  'R Tewatia',
  'A Manohar',
  'Rashid Khan',
  'LH Ferguson',
  'Mohammed Shami',
  'VR Aaron']}

In [31]:
eg_file['info']['outcome']

{'winner': 'Gujarat Titans', 'by': {'wickets': 5}}

In [27]:
len(eg_file['innings'])

2

In [30]:
eg_file['innings'][1]['target']

{'overs': 20, 'runs': 159}

In [32]:
eg_file['innings'][1].keys()

dict_keys(['team', 'overs', 'powerplays', 'target'])

In [69]:
eg_file['innings'][0].keys()


dict_keys(['team', 'overs', 'powerplays'])

In [39]:
#runs scored by second team
runs = 0 # shld be above 159
wickets = 0 # shld be 5
for over in eg_file['innings'][1]['overs']:
    for delivery in over['deliveries']:
        runs += delivery['runs']['total']
        if 'wickets' in delivery:
            wickets += 1
print(runs,'/', wickets)


161 / 5


### the first task is to record each and every innings played by a player in the IPL

In [5]:
# Getting no of matches from player registry of each match
n = 0
inn = 0
pl_name = 'V Kohli'
for file in os.listdir(json_files):
    if file.endswith('json'):
        n+=1
        with open(json_files+f'/{file}') as json_file:
            match = json.loads(json_file.read())
            if pl_name in match['info']['registry']['people']:
                inn += 1
print(f'Total no. of matches: {n}')
print(f'Total No. of matches played by {pl_name}: {inn} ')


Total no. of matches: 950
Total No. of matches played by V Kohli: 222 


In [6]:
# Getting No. of matches from line-ups
n = 0
inn = 0
pl_name = 'V Kohli'
for file in os.listdir(json_files):
    if file.endswith('json'):
        n += 1
        with open(json_files+f'/{file}') as json_file:
            match = json.loads(json_file.read())
            if 'Royal Challengers Bangalore' in match['info']['players']:
                if 'V Kohli' in match['info']['players']['Royal Challengers Bangalore']:
                    inn += 1
print(f'Total no. of matches: {n}')
print(f'Total No. of matches played by {pl_name}: {inn} ')


Total no. of matches: 950
Total No. of matches played by V Kohli: 222 


In [7]:
n = 0
mats = 0
left_out = 0
inns = []
pl_name = 'V Kohli'
for file in os.listdir(json_files):
    if file.endswith('json'):
        n += 1
        with open(json_files+f'/{file}') as json_file:
            match = json.loads(json_file.read())
            if 'Royal Challengers Bangalore' in match['info']['players']:
                if pl_name in match['info']['players']['Royal Challengers Bangalore']:
                    mats += 1
                else:
                    left_out += 1
print(f'Total no. of matches: {n}')
print(f'Total No. of matches played by {pl_name}: {mats} ')
print(f'RCB matches where {pl_name} did not play: {left_out}')

Total no. of matches: 950
Total No. of matches played by V Kohli: 222 
RCB matches where V Kohli did not play: 4


### Getting all the runs scored by V Kohli

In [73]:
runs = []
team_name = 'Royal Challengers Bangalore'
pl_name = 'V Kohli'
for file in os.listdir(json_files):
    if file.endswith('json'):
        with open(json_files+'/'+file) as f:
            match = json.loads(f.read())
            match_runs = 0
            if pl_name in match['info']['registry']['people']:
                for inn in match['innings']:
                    if team_name == inn['team']:
                        for over in inn['overs']:
                            for delivery in over['deliveries']:
                                if pl_name == delivery['batter']:
                                    match_runs += delivery['runs']['batter']
                runs.append(match_runs)


In [74]:
len(runs)

211

In [77]:
np.mean(runs)

30.33175355450237

In [78]:
np.median(runs)

23.0

In [79]:
np.std(runs)

26.387090881176956

In [80]:
max(runs), min(runs)

(113, 0)

In [75]:
sum(runs)

6400

Getting all runs scored by V Kohli and storing them season wise

In [88]:
runs = {}
team_name = 'Royal Challengers Bangalore'
pl_name = 'V Kohli'
for file in os.listdir(json_files):
    if file.endswith('json'):
        with open(json_files+'/'+file) as f:
            match = json.loads(f.read())
            match_runs = 0
            season = str(match['info']['season'])
            if pl_name in match['info']['registry']['people']:
                for inn in match['innings']:
                    if team_name == inn['team']:
                        for over in inn['overs']:
                            for delivery in over['deliveries']:
                                if pl_name == delivery['batter']:
                                    match_runs += delivery['runs']['batter']
                runs.setdefault(season, [])
                runs[season].append(match_runs)


In [89]:
# Highest score in every season
for season in runs:
    print(season,':', max(runs[season]))

2017 : 64
2018 : 92
2019 : 100
2020/21 : 90
2021 : 72
2022 : 48
2007/08 : 38
2009 : 50
2009/10 : 58
2011 : 71
2012 : 73
2013 : 99
2014 : 73
2015 : 82
2016 : 113


In [90]:
# Total runs in every season
for season in runs:
    print(season,':', sum(runs[season]))

2017 : 308
2018 : 530
2019 : 464
2020/21 : 471
2021 : 405
2022 : 107
2007/08 : 165
2009 : 246
2009/10 : 307
2011 : 557
2012 : 364
2013 : 639
2014 : 359
2015 : 505
2016 : 973


### Rohit

In [66]:
n = 0
mats = 0
left_out = 0
inns = []
team_name = 'Mumbai Indians'
team_name2 = 'Deccan Chargers'
pl_name = 'RG Sharma'
for file in os.listdir(json_files):
    if file.endswith('json'):
        n += 1
        with open(json_files+f'/{file}') as json_file:
            match = json.loads(json_file.read())
            # if type(match['info']['season']) is str:
            #     season = int(match['info']['season'].split('/')[0])
            # else: 
            #     season = match['info']['season']
            # if season > 2009:
        
            if team_name in match['info']['players'] :
                if pl_name in match['info']['players'][team_name]:
                    mats += 1
            elif team_name2 in match['info']['players'] :
                if pl_name in match['info']['players'][team_name2]:
                    mats += 1
            else:
                left_out += 1
print(f'Total no. of matches: {n}')
print(f'Total No. of matches played by {pl_name}: {mats} ')
print(f'{team_name} matches where {pl_name} did not play: {left_out}')


Total no. of matches: 898
Total No. of matches played by RG Sharma: 211 
Mumbai Indians matches where RG Sharma did not play: 612


In [93]:
runs = {}
# team_name = 'Royal Challengers Bangalore'
pl_name = 'RG Sharma'
for file in os.listdir(json_files):
    if file.endswith('json'):
        with open(json_files+'/'+file) as f:
            match = json.loads(f.read())
            match_runs = 0
            season = str(match['info']['season'])
            if pl_name in match['info']['registry']['people']:
                for team in match['info']['teams']:
                    if pl_name in match['info']['players'][team]:
                        team_name = team
                for inn in match['innings']:
                    if team_name == inn['team']:
                        for over in inn['overs']:
                            for delivery in over['deliveries']:
                                if pl_name == delivery['batter']:
                                    match_runs += delivery['runs']['batter']
                runs.setdefault(season, [])
                runs[season].append(match_runs)


In [94]:
# Total runs in every season
for season in runs:
    print(season, ':', sum(runs[season]))


2017 : 333
2018 : 286
2019 : 405
2020/21 : 334
2021 : 381
2022 : 80
2007/08 : 404
2009 : 362
2009/10 : 404
2011 : 372
2012 : 433
2013 : 538
2014 : 390
2015 : 482
2016 : 489


In [100]:
# Highest score in every season
for season in runs:
    print(season, ':', max(runs[season]))


2017 : 67
2018 : 94
2019 : 67
2020/21 : 80
2021 : 63
2022 : 41
2007/08 : 76
2009 : 52
2009/10 : 73
2011 : 87
2012 : 109
2013 : 79
2014 : 59
2015 : 98
2016 : 85


In [98]:
# seeing how consistent he has been by looking at std
for season in runs:
    print(season, ':', round(np.std(runs[season]), 2))


2017 : 22.03
2018 : 25.36
2019 : 18.75
2020/21 : 28.11
2021 : 15.82
2022 : 14.71
2007/08 : 24.52
2009 : 14.77
2009/10 : 24.6
2011 : 24.87
2012 : 30.53
2013 : 25.16
2014 : 17.8
2015 : 23.32
2016 : 30.18


In [101]:
flat_runs = []
for season in runs:
    flat_runs.extend(runs[season])

In [103]:
np.mean(flat_runs), np.median(flat_runs)

(26.235023041474655, 19.0)

### Extract Data

In [8]:
def process_season(season):
    if '/' in str(season):
        t = {'2020/21':2020, '2007/08':2008, '2009/10':2010}
        return t[season]
    else:
        return int(season)

In [15]:
def get_player_scores(pl_name):
    runs = {}
    for file in os.listdir(json_files):
        if file.endswith('json'):
            with open(json_files+'/'+file) as f:
                match = json.loads(f.read())
                match_runs = 0
                team_name = ''
                season = process_season((match['info']['season']))
                if pl_name in match['info']['registry']['people']:
                    for team in match['info']['teams']:
                        if pl_name in match['info']['players'][team]:
                            team_name = team
                    for inn in match['innings']:
                        if team_name == inn['team']:
                            for over in inn['overs']:
                                for delivery in over['deliveries']:
                                    if pl_name == delivery['batter']:
                                        match_runs += delivery['runs']['batter']
                    runs.setdefault(season, [])
                    runs[season].append(match_runs)
    return runs


In [16]:
get_player_scores('RG Sharma')

{2017: [3, 2, 4, 0, 40, 0, 5, 58, 5, 56, 10, 67, 5, 27, 1, 26, 24],
 2018: [15, 11, 18, 94, 0, 2, 56, 0, 24, 11, 36, 0, 6, 13],
 2019: [14, 48, 32, 13, 11, 47, 28, 30, 5, 67, 12, 24, 55, 4, 15],
 2020: [12, 4, 70, 80, 35, 11, 35, 5, 6, 8, 0, 68],
 2021: [19, 43, 32, 44, 63, 14, 35, 18, 22, 33, 8, 43, 7],
 2022: [41, 10, 3, 26],
 2008: [0, 66, 36, 0, 76, 17, 23, 5, 33, 35, 6, 50, 57],
 2009: [36, 52, 3, 18, 17, 38, 21, 38, 9, 9, 6, 32, 42, 12, 5, 24],
 2010: [13, 19, 1, 45, 49, 45, 2, 11, 73, 6, 8, 51, 68, 11, 2, 0],
 2011: [27, 0, 0, 20, 87, 56, 13, 18, 12, 49, 5, 4, 58, 10, 0, 13],
 2012: [0, 1, 73, 21, 29, 2, 50, 12, 42, 3, 60, 0, 109, 5, 12, 0, 14],
 2013: [11, 8, 74, 62, 2, 73, 34, 10, 79, 22, 39, 16, 37, 20, 14, 25, 8, 2, 2],
 2014: [27, 2, 50, 4, 1, 39, 59, 19, 14, 51, 40, 18, 30, 16, 20],
 2015: [98, 0, 0, 50, 42, 30, 24, 27, 26, 46, 18, 15, 30, 7, 19, 50],
 2016: [7, 84, 7, 5, 62, 65, 0, 68, 85, 5, 25, 15, 31, 30]}

the snippet below is absolutely unfeasible since it seperately extracts info for each player and therefore goes over the 900 or so files almost 500 times

In [11]:
# get_all_player_scores
# all_players_runs = {}
# for file in os.listdir(json_files):
#     if file.endswith('json'):
#         with open(json_files+'/'+file) as f:
#             match = json.loads(f.read())
#             for team in match['info']['players']:
#                 for player in match['info']['players'][team]:
#                     if player not in all_players_runs:
#                         all_players_runs[player] = get_player_scores(player)
#                     else:
#                         continue

In [17]:
# # extract the runs scored by all the players that have ever played in the competition
# all_players_runs = {}
# for file in os.listdir(json_files):
#     if file.endswith('json'):
#         with open(json_files+'/'+file) as f:
#             match = json.loads(f.read())
#             match_runs = {}
#             season = process_season(match['info']['season'])
#             for inn in match['innings']:
#                 for over in inn['overs']:
#                     for delivery in over['deliveries']:
#                         match_runs.setdefault(delivery['batter'], [])
#                         match_runs[delivery['batter']].append(delivery['runs']['batter'])
#     for player in match_runs:
#         all_players_runs.setdefault(player, {})
#         all_players_runs[player].setdefault(season, [])
#         all_players_runs[player][season].append(sum(match_runs[player]))



In [9]:
# every player's season wise- runs, balls,dots, out, 4s, 6s
# extract the runs scored by all the players that have ever played in the competition 
# along with the number of balls they faced
# also add the boundaries they hit
all_players_runs = {}
for file in os.listdir(json_files):
    if file.endswith('json'):
        with open(json_files+'/'+file) as f:
            match = json.loads(f.read())
            match_runs = {}
            dismissals = set()
            fours = {}
            sixes = {}
            dots = {}
            balls = {}
            season = process_season(match['info']['season'])
            
            for inn in match['innings']:
                for over in inn['overs']:
                    for delivery in over['deliveries']:
                        fours.setdefault(delivery['batter'], 0)
                        sixes.setdefault(delivery['batter'], 0)
                        dots.setdefault(delivery['batter'], 0)
                        balls.setdefault(delivery['batter'], 0)
                        if 'wickets' in delivery:
                            dismissals.add(delivery['wickets'][0]['player_out'])
                        if delivery['runs']['batter'] == 4:
                            fours[delivery['batter']] += 1
                        if delivery['runs']['batter'] == 6:
                            sixes[delivery['batter']] += 1
                        if 'extras' not in delivery or ( ('wides' not in delivery['extras']) and ('noballs' not in delivery['extras'])):
                            balls[delivery['batter']] += 1
                            if delivery['runs']['batter'] == 0:
                                dots[delivery['batter']] += 1

                        match_runs.setdefault(delivery['batter'], [])
                        match_runs[delivery['batter']].append(delivery['runs']['batter'])
    for player in match_runs:
        if player in dismissals:
            out = 1
        else:
            out = 0
        all_players_runs.setdefault(player, {})
        all_players_runs[player].setdefault(season, [])
        all_players_runs[player][season].append(
            {
                'Runs':sum(match_runs[player]),
                'Balls': balls[player],
                'Dots': dots[player],
                'Out':out,
                '4s': fours[player],
                '6s': sixes[player]
            })

In [10]:
def career(pl_name):
    thirties = 0
    fifties = 0
    hundreds = 0
    SR = []
    headers = ['Inns']+list(all_players_runs['V Kohli'][2016][1].keys())+['SR','Avg','30s', '50s', '100s']
    pl_career = {}
    for season in all_players_runs[pl_name]:
        for header in headers:
            pl_career.setdefault(header, 0)
        for match in all_players_runs[pl_name][season]:
            pl_career['Inns']+=1
            for stat in match:
                pl_career[stat]+=match[stat]
            if match['Balls'] > 0:
                SR.append((match['Runs']/match['Balls'])*100)
            run = match['Runs']
            if run in range(30,50):
                pl_career['30s'] += 1
            elif run in range(50, 100):
                pl_career['50s'] += 1
            elif run >= 100:
                pl_career['100s'] += 1

    pl_career['SR'] = round((pl_career['Runs']/pl_career['Balls'])*100,2)
    if pl_career['Out'] == 0:
        pl_career['Avg'] = 0
    else:
        pl_career['Avg'] = round(pl_career['Runs']/pl_career['Out'], 2)
    return pl_career

In [13]:
all_players_runs['V Kohli'][2022]

[{'Runs': 41, 'Balls': 29, 'Dots': 5, 'Out': 0, '4s': 1, '6s': 2},
 {'Runs': 12, 'Balls': 7, 'Dots': 3, 'Out': 1, '4s': 2, '6s': 0},
 {'Runs': 5, 'Balls': 6, 'Dots': 2, 'Out': 1, '4s': 0, '6s': 0},
 {'Runs': 48, 'Balls': 36, 'Dots': 10, 'Out': 1, '4s': 5, '6s': 0},
 {'Runs': 1, 'Balls': 3, 'Dots': 2, 'Out': 1, '4s': 0, '6s': 0},
 {'Runs': 12, 'Balls': 14, 'Dots': 6, 'Out': 1, '4s': 1, '6s': 0},
 {'Runs': 0, 'Balls': 1, 'Dots': 1, 'Out': 1, '4s': 0, '6s': 0},
 {'Runs': 0, 'Balls': 1, 'Dots': 1, 'Out': 1, '4s': 0, '6s': 0},
 {'Runs': 9, 'Balls': 10, 'Dots': 7, 'Out': 1, '4s': 2, '6s': 0},
 {'Runs': 58, 'Balls': 53, 'Dots': 20, 'Out': 1, '4s': 6, '6s': 1},
 {'Runs': 30, 'Balls': 33, 'Dots': 18, 'Out': 1, '4s': 3, '6s': 1},
 {'Runs': 0, 'Balls': 1, 'Dots': 1, 'Out': 1, '4s': 0, '6s': 0},
 {'Runs': 20, 'Balls': 14, 'Dots': 7, 'Out': 1, '4s': 2, '6s': 1},
 {'Runs': 73, 'Balls': 54, 'Dots': 16, 'Out': 1, '4s': 8, '6s': 2},
 {'Runs': 25, 'Balls': 24, 'Dots': 10, 'Out': 1, '4s': 2, '6s': 0},
 {

In [15]:
career('V Kohli')

{'Inns': 215,
 'Runs': 6634,
 'Balls': 5115,
 'Dots': 1745,
 'Out': 183,
 '4s': 581,
 '6s': 219,
 'SR': 129.7,
 'Avg': 36.25,
 '30s': 46,
 '50s': 45,
 '100s': 5}

### Create a CSV File with every player and their career records

In [184]:
# with open('ipl_batting_career.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     headers = ['Name']+list(career('V Kohli').keys())
#     writer.writerow(headers)
#     for player in all_players_runs:
#         writer.writerow((player,)+tuple(career(player).values()))


### Create a JSON file with every player and their career as well as Season records

In [16]:
def get_season_stats (pl_name, season):
    season_stats = {}

    for header in career(pl_name):
        season_stats[header] = 0

    for match in all_players_runs[pl_name][season]:
        season_stats['Inns'] += 1
        for stat in match:
            season_stats[stat] += match[stat]
        run = match['Runs']
        if run in range(30,50):
            season_stats['30s'] += 1
        elif run in range(50, 100):
            season_stats['50s'] += 1
        elif run >= 100:
            season_stats['100s'] += 1
    try:
        season_stats['SR'] = round((season_stats['Runs']/season_stats['Balls'])*100,2)
        season_stats['Avg'] = round(season_stats['Runs']/season_stats['Out'], 2)
    except ZeroDivisionError:
        season_stats['SR'] = 0
        season_stats['Avg'] = 0

    return season_stats

In [17]:
seasons = range(2008, 2023)
player_batting_stats = {}
for player in all_players_runs:
    player_batting_stats[player] = {}
    player_batting_stats[player]['career'] = career(player)
    player_batting_stats[player]['seasons'] = {}
    for season in seasons:
        if season not in all_players_runs[player]:
            continue
        player_batting_stats[player]['seasons'][season] = {}
        player_batting_stats[player]['seasons'][season]['matches'] = all_players_runs[player][season]
        player_batting_stats[player]['seasons'][season]['stats'] = get_season_stats(player, season)

In [21]:
player_batting_stats['RG Sharma']['seasons'][2022]

{'matches': [{'Runs': 41, 'Balls': 32, 'Dots': 14, 'Out': 1, '4s': 4, '6s': 2},
  {'Runs': 10, 'Balls': 5, 'Dots': 2, 'Out': 1, '4s': 0, '6s': 1},
  {'Runs': 3, 'Balls': 12, 'Dots': 9, 'Out': 1, '4s': 0, '6s': 0},
  {'Runs': 26, 'Balls': 15, 'Dots': 8, 'Out': 1, '4s': 4, '6s': 1},
  {'Runs': 28, 'Balls': 17, 'Dots': 9, 'Out': 1, '4s': 3, '6s': 2},
  {'Runs': 6, 'Balls': 7, 'Dots': 4, 'Out': 1, '4s': 1, '6s': 0},
  {'Runs': 0, 'Balls': 2, 'Dots': 2, 'Out': 1, '4s': 0, '6s': 0},
  {'Runs': 39, 'Balls': 31, 'Dots': 14, 'Out': 1, '4s': 5, '6s': 1},
  {'Runs': 2, 'Balls': 5, 'Dots': 3, 'Out': 1, '4s': 0, '6s': 0},
  {'Runs': 43, 'Balls': 28, 'Dots': 12, 'Out': 1, '4s': 5, '6s': 2},
  {'Runs': 2, 'Balls': 6, 'Dots': 5, 'Out': 1, '4s': 0, '6s': 0},
  {'Runs': 18, 'Balls': 14, 'Dots': 8, 'Out': 1, '4s': 4, '6s': 0},
  {'Runs': 48, 'Balls': 35, 'Dots': 15, 'Out': 1, '4s': 2, '6s': 4},
  {'Runs': 2, 'Balls': 13, 'Dots': 11, 'Out': 1, '4s': 0, '6s': 0}],
 'stats': {'Inns': 14,
  'Runs': 268,
  'B

#### Store the batting stats in a JSON file

In [19]:
# with open('ipl_batting_stats.json', 'w') as f:
#     f.write(json.dumps(player_batting_stats))

#### Read the Batting stats from a JSON file

In [6]:
with open('ipl_batting_stats.json') as f:
    batting_stats = json.loads(f.read())

In [28]:
player_batting_stats['MS Dhoni']['seasons'][2022]

{'matches': [{'Runs': 50, 'Balls': 37, 'Dots': 16, 'Out': 0, '4s': 7, '6s': 1},
  {'Runs': 16, 'Balls': 6, 'Dots': 2, 'Out': 0, '4s': 2, '6s': 1},
  {'Runs': 23, 'Balls': 27, 'Dots': 13, 'Out': 1, '4s': 1, '6s': 1},
  {'Runs': 3, 'Balls': 6, 'Dots': 3, 'Out': 1, '4s': 0, '6s': 0},
  {'Runs': 28, 'Balls': 13, 'Dots': 0, 'Out': 0, '4s': 3, '6s': 1},
  {'Runs': 12, 'Balls': 8, 'Dots': 4, 'Out': 1, '4s': 1, '6s': 1},
  {'Runs': 8, 'Balls': 7, 'Dots': 3, 'Out': 1, '4s': 1, '6s': 0},
  {'Runs': 2, 'Balls': 3, 'Dots': 1, 'Out': 1, '4s': 0, '6s': 0},
  {'Runs': 21, 'Balls': 8, 'Dots': 2, 'Out': 0, '4s': 1, '6s': 2},
  {'Runs': 36, 'Balls': 33, 'Dots': 21, 'Out': 0, '4s': 4, '6s': 2},
  {'Runs': 7, 'Balls': 10, 'Dots': 5, 'Out': 1, '4s': 0, '6s': 0},
  {'Runs': 26, 'Balls': 28, 'Dots': 12, 'Out': 1, '4s': 1, '6s': 1}],
 'stats': {'Inns': 12,
  'Runs': 232,
  'Balls': 186,
  'Dots': 82,
  'Out': 7,
  '4s': 21,
  '6s': 10,
  'SR': 124.73,
  'Avg': 33.14,
  '30s': 1,
  '50s': 1,
  '100s': 0}}

### Store the player stats in a python file

In [22]:
with open('ipl_batting_stats.py', 'w', newline='') as f:
    f.write(f'player_stats = {player_batting_stats}')